# Environment

These opcodes give you access to the Ethereum environment

#### Address

Returns the address of the account currently executing this program

In [ ]:
pub fn address(evm: &mut Evm) {
    evm.stack.push(evm.sender);
    evm.pc += 1;
    evm.gas_dec(3);
}

#### Balance

Get the balance of the given address. We only mock this, so we always return the same value for every address.

In [ ]:
pub fn balance(evm: &mut Evm) {
    let _address = evm.stack.pop();
    evm.stack.push(999_999_999_99u128);

    evm.pc += 1;
    evm.gas_dec(2600); // 100 if warm
}

#### Origin

The address that originally triggered the execution. This is `tx.origin`in Solidity.
For us `tx.origin` is always equal to `msg.sender`. That is why we simply return the `sender`.

In [ ]:
pub fn origin(evm: &mut Evm) {
    evm.stack.push(evm.sender);
    evm.pc += 1;
    evm.gas_dec(2);
}

#### Caller

In [ ]:
pub fn caller(evm: &mut Evm) {
    evm.stack.push(Address::from_str("0x414b60745072088d013721b4a28a0559b1A9d213").unwrap());
    evm.pc += 1;
    evm.gas_dec(2);
}

#### Callvalue

Returns the value of Ether (wei) provided for this execution

In [ ]:
pub fn callvalue(evm: &mut EVM) {
    evm.stack.push(evm.value);
    evm.pc += 1;
    evm.gas_dec(2);
}

#### Calldataload

Pushes the current input data (32 bytes) on the stack

In [ ]:
pub fn calldataload(evm: &mut Evm) {
    let i = evm.stack.pop();

    let mut delta = 0;
    if i + 32 > evm.calldata.len() {
        delta = i + 32 - evm.calldata.len();
    }

    // always has to be 32 bytes
    // if it's not, append 0x00 bytes until it is
    let mut calldata = evm.calldata[i..i + 32 - delta].to_vec();
    calldata.extend(vec![0x00; delta]);

    evm.stack.push(calldata);
    evm.pc += 1;
    evm.gas_dec(3);
}

#### Calldatasize

Pushes the size of the calldata on the stack

In [ ]:
pub fn calldatasize(evm: &mut EVM) {
    evm.stack.push(evm.calldata.len());
    evm.pc += 1;
    evm.gas_dec(2);
}

#### Calldatacopy

Stores a specified part of the calldata in memory

In [ ]:
pub fn calldatacopy(evm: &mut EVM) {
    let dest_offset = evm.stack.pop();
    let offset = evm.stack.pop();
    let size = evm.stack.pop();

    let calldata = &evm.calldata[offset..offset + size];
    let memory_expansion_cost = evm.memory.store(dest_offset, calldata);

    let static_gas = 3;
    let minimum_word_size = (size + 31) / 32;
    let dynamic_gas = 3 * minimum_word_size + memory_expansion_cost;

    evm.gas_dec(static_gas + dynamic_gas);
    evm.pc += 1;
}

#### Codesize

Puts the size of the currently running program on the stack

In [ ]:
pub fn codesize(evm: &mut EVM) {
    evm.stack.push(evm.program.len());
    evm.pc += 1;
    evm.gas_dec(2);
}

#### Codecopy

Stores a specified part of the program in memory

In [ ]:
pub fn codecopy(evm: &mut EVM) {
    let dest_offset = evm.stack.pop();
    let offset = evm.stack.pop();
    let size = evm.stack.pop();

    let code = &evm.program[offset..offset + size];
    let memory_expansion_cost = evm.memory.store(dest_offset, code);

    let static_gas = 3;
    let minimum_word_size = (size + 31) / 32;
    let dynamic_gas = 3 * minimum_word_size + memory_expansion_cost;

    evm.gas_dec(static_gas + dynamic_gas);
    evm.pc += 1;
}

#### Gas Price

The current gas price. Because we are running everything locally, the gas price is simply 0.

In [ ]:
pub fn gasprice(evm: &mut EVM) {
    evm.stack.push(0x00);
    evm.pc += 1;
    evm.gas_dec(2);
}

#### External Code Size

The size of another program given by its address. There are no other programs in our simplified world so we simply return 0.

In [ ]:
pub fn extcodesize(evm: &mut EVM) {
    let address = evm.stack.pop();
    evm.stack.push(0x00);
    evm.gas_dec(2600); // 100 if warm
    evm.pc += 1;
}

#### External Code Copy

Stores a specified part of another program in memory

In [ ]:
pub fn extcodecopy(evm: &mut EVM) {
    let address = evm.stack.pop();
    let dest_offset = evm.stack.pop();
    let offset = evm.stack.pop();
    let size = evm.stack.pop();

    let extcode: Vec<u8> = vec![]; // no external code
    let memory_expansion_cost = evm.memory.store(dest_offset, &extcode);

    // refactor this in separate method
    let minimum_word_size = (size + 31) / 32;
    let dynamic_gas = 3 * minimum_word_size + memory_expansion_cost;

    let address_access_cost = if evm.is_warm(address) { 100 } else { 2600 };

    evm.gas_dec(dynamic_gas + address_access_cost);
    evm.pc += 1;
}

#### Return Data Size

Get size of output data from the previous call from the current environment. As our execution is the only one running, there is no previous return data. Therefore we can simply return 0.

In [ ]:
pub fn returndatasize(evm: &mut EVM) {
    evm.stack.push(0x00); // no return data
    evm.pc += 1;
    evm.gas_dec(2);
}

#### Return Data Copy

Stores a specified part of the previous return data in memory

In [ ]:
pub fn returndatacopy(evm: &mut EVM) {
    let dest_offset = evm.stack.pop();
    let offset = evm.stack.pop();
    let size = evm.stack.pop();

    let returndata = &evm.program[offset..offset + size];
    let memory_expansion_cost = evm.memory.store(dest_offset, returndata);

    let minimum_word_size = (size + 31) / 32;
    let dynamic_gas = 3 * minimum_word_size + memory_expansion_cost;

    evm.gas_dec(3 + dynamic_gas);
    evm.pc += 1;
}

#### External Code Hash

The hash of another program given by its address. There are no other programs in our simplified world so we simply return 0.

In [ ]:
pub fn extcodehash(evm: &mut EVM) {
    let address = evm.stack.pop();
    evm.stack.push(0x00); // no code

    evm.gas_dec(2600); // 100 if warm
    evm.pc += 1;
}

#### Block Hash

Get the hash of one of the 256 most recent complete blocks and push it on the stack.

In [ ]:
pub fn blockhash(evm: &mut EVM) {
    let block_number = evm.stack.pop();
    if block_number > 256 {
        panic!("Only last 256 blocks can be accessed");
    }
    evm.stack.push(0x1cbcfa1ffb1ca1ca8397d4f490194db5fc0543089b9dee43f76cf3f962a185e8);
    evm.pc += 1;
    evm.gas_dec(20);
}

#### Coinbase

Get the address of the miner for this block

In [ ]:
pub fn coinbase(evm: &mut EVM) {
    evm.stack.push(0x1cbcfa1ffb1ca1ca8397d4f490194db5fc0543089b9dee43f76cf3f962a185e8);
    evm.pc += 1;
    evm.gas_dec(2);
}